In [1]:
import importlib

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.utils import shuffle
import torch
import utils3
import model3
importlib.reload(utils3)
importlib.reload(model3)
from utils3 import *
from model3 import *
tf.test.is_built_with_cuda()

False

In [2]:
dic = get_dict_icon_sketches()
icons, icons_name_cat = load_icons(dic)
sketches, sketch_names_array = load_sketches(dic)

sketch_icon_indices, positive_labels = create_positive_sketch_icon_indices(icons_name_cat, sketch_names_array)
print(f'We have {len(sketch_icon_indices)} samples.')
# shuffle the indices and labels
sketch_icon_indices, positive_labels = shuffle(sketch_icon_indices, positive_labels)

We have 1485 samples.


In [11]:
NEGATIVE_PAIRS_LEN = 1

def shuffle_in_unison(a, b):
     n_elem = a.shape[0]
     indeces = np.random.permutation(n_elem)
     return a[indeces], b[indeces]

class Dataset():
    def __init__(self, indices, labels):
        self.indices = indices
        self.labels = labels

    def __len__(self):
        # Return the size of the dataset
        return len(self.indices)

    def __getitem__(self, index):
        # Retrieve indices and labels at the given index
        indices = self.indices[index]
        label = self.labels[index]

        return indices, label

def create_datasets(sketch_icon_indices, labels, dataset_class, p_train=0.9, p_test=0.1):
    len_data = len(sketch_icon_indices)

    # Define partition sizes
    num_train = int(np.ceil(len_data*p_train))
    num_test = int(np.floor(len_data*p_test))

    sketch_icon_indices_Train, positive_labels_Train = sketch_icon_indices[:num_train], labels[:num_train]
    sketch_icon_indices_Test, labels_test = sketch_icon_indices[-num_test:], labels[-num_test:]

    negative_indices, negative_labels_Train = create_negative_sketch_icon_indices(sketch_icon_indices_Train, NEGATIVE_PAIRS_LEN)
    #print(f'We have {len(sketch_icon_indices_Train)} positive pair samples in the training set.')
    #print(f'We have {len(negative_indices)} negative pair samples in the training set.')

    negative_array = np.array(negative_indices)
    # This array includes the posititive and negative pair indices
    pair_indices_Train = np.concatenate((sketch_icon_indices_Train, negative_array), axis=0)
    labels_Train = np.concatenate((positive_labels_Train, negative_labels_Train), axis=0)
    pair_indices_Train, labels_Train = shuffle_in_unison(pair_indices_Train, labels_Train)

    training_set = dataset_class(pair_indices_Train, labels_Train)
    test_set = dataset_class(sketch_icon_indices_Test, labels_test)
    return training_set, test_set

training_set, test_set = create_datasets(sketch_icon_indices, positive_labels, Dataset)   
print(f'We have {len(training_set)} samples in the training set.')
print(f'We have {len(test_set)} samples in the test set.')


[[  80    2]
 [1341   49]
 [1398   26]
 ...
 [1058   39]
 [ 936   24]
 [1050   17]]
We have 2674 samples in the training set.
We have 148 samples in the test set.


In [5]:
batch_size = 128

def load_dataset():

    trainloader = torch.utils.data.DataLoader(
      dataset = training_set,
      batch_size= batch_size,
      num_workers = 0,
      shuffle= False
    )

    testloader = torch.utils.data.DataLoader(
      dataset = test_set,
      batch_size= batch_size,
      num_workers= 0,
      shuffle= False
    )

    return trainloader, testloader

trainloader, testloader = load_dataset()

In [6]:
num_epochs = 20
margin = 1
optimizer = tf.keras.optimizers.Adam(0.0001)
tf.keras.backend.set_floatx('float32')
siameseModel = googlenet()

loss = siamese_loss

def train_step( icons, sketches , labels, margin):
    with tf.GradientTape() as tape:
        model1 = siameseModel(icons, training = True)
        #print(model1)
        model2 = siameseModel(sketches, training = True)    
        tape.watch(model1)
        tape.watch(model2)
        labels = tf.convert_to_tensor(labels, dtype=tf.float32)
        tape.watch(labels)
        current_loss = loss(model1, model2, labels)
    grads = tape.gradient(current_loss, siameseModel.trainable_variables)
    optimizer.apply_gradients(zip(grads, siameseModel.trainable_variables))
    return current_loss

In [7]:
test_icon_indeces = set()
test_sketch_indices = set()
for s, i in test_set.indices:
    test_sketch_indices.add(s)
    test_icon_indeces.add(i)

test_sketch_list = []
test_sketch_name = []
for s in test_sketch_indices:
    test_sketch_list.append(sketches[s])
    test_sketch_name.append(sketch_names_array[s])
test_sketches = np.array(test_sketch_list)
print(test_sketches.shape)

test_icon_list = []
test_icon_name = []
for i in test_icon_indeces:
    test_icon_list.append(icons[i])
    test_icon_name.append(icons_name_cat[i][0])
test_icons = np.array(test_icon_list)
print(test_icons.shape)


(148, 224, 224, 3)
(52, 224, 224, 3)


In [8]:

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    training_set, test_set = create_datasets(sketch_icon_indices, positive_labels, Dataset)
    trainloader, testloader = load_dataset()
    for _,(indices, labels) in enumerate(trainloader):
        i, s = get_batch(indices, icons, sketches)
        loss_value = train_step(i, s, labels, margin)
        epoch_loss_avg.update_state(loss_value)
    print("Epoch {:d}: Loss: {:.3f}".format(epoch,epoch_loss_avg.result()))
    if epoch%1==0:
        acc_1 = 0
        acc_10 = 0
        sketch_representations = []
        for i in range(len(test_sketches)):
            sketch_repr =  siameseModel(np.expand_dims(test_sketches[i], 0), training = False)
            sketch_representations.append(sketch_repr)
        sketch_representations = np.vstack(sketch_representations)

        icon_representations = []
        for i in range(len(test_icons)):
            icon_repr =  siameseModel(np.expand_dims(test_icons[i], 0), training = False)
            icon_representations.append(icon_repr)
        icon_representations = np.vstack(icon_representations)

        for i in range(len(sketch_representations)):
            sketch_repr = sketch_representations[i]
            sketch_representations_tile = np.tile(sketch_repr, len(test_icons)).reshape(len(test_icons), 256)
            diff = np.sqrt(np.mean((sketch_representations_tile - icon_representations)**2, -1))
            top_k = np.argsort(diff)[:10]
            
            for j in range(len(top_k)):
                index = top_k[j]
                if j == 0 and test_sketch_name[i] == test_icon_name[index]:
                    acc_1 = acc_1 + 1
                    acc_10 = acc_10 + 1
                    break
                elif test_sketch_name[i] == test_icon_name[index]:
                    acc_10 = acc_10 + 1
                    break
        print("Accuracy of top 1: " + str(acc_1/len(test_sketches)))
        print("Accuracy of top 10: " + str(acc_10/len(test_sketches)))


Epoch 0: Loss: 5.208
Accuracy of top 1: 0.02027027027027027
Accuracy of top 10: 0.18243243243243243
Epoch 1: Loss: 4.967
Accuracy of top 1: 0.013513513513513514
Accuracy of top 10: 0.20270270270270271
Epoch 2: Loss: 4.915
Accuracy of top 1: 0.013513513513513514
Accuracy of top 10: 0.20270270270270271
Epoch 3: Loss: 4.918
Accuracy of top 1: 0.013513513513513514
Accuracy of top 10: 0.1891891891891892
Epoch 4: Loss: 4.972
Accuracy of top 1: 0.013513513513513514
Accuracy of top 10: 0.1891891891891892
Epoch 5: Loss: 4.956
Accuracy of top 1: 0.013513513513513514
Accuracy of top 10: 0.21621621621621623
Epoch 6: Loss: 4.951
Accuracy of top 1: 0.013513513513513514
Accuracy of top 10: 0.1891891891891892
Epoch 7: Loss: 4.912
Accuracy of top 1: 0.013513513513513514
Accuracy of top 10: 0.19594594594594594
Epoch 8: Loss: 4.943
Accuracy of top 1: 0.013513513513513514
Accuracy of top 10: 0.19594594594594594
Epoch 9: Loss: 4.910
Accuracy of top 1: 0.013513513513513514
Accuracy of top 10: 0.195945945945